# Prediction of Our ANN Model:

In [1]:
import tensorflow as tf
import keras
from keras.models import load_model
import pickle
import numpy as np
import pandas as pd

## Let's check our orginial Data first:

In [2]:
churn = pd.read_csv('datasets/Churn_Modelling.csv')
churn.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)
churn.index = churn.index + 1

In [3]:
churn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9996,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9997,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9998,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9999,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


# Loading the ANN trained model, scaler file and encoder file:

In [4]:
model = load_model('model.keras')
print(model)

<Sequential name=sequential_10, built=True>


In [5]:
with open('scaler.pkl','rb') as f:
    scaler = pickle.load(f)
with open('geography_label.pkl','rb') as f:
    geo_label = pickle.load(f)
with open('gender_label.pkl','rb') as f:
    gender_label = pickle.load(f)

In [6]:
scaler,geo_label,gender_label

(StandardScaler(), OneHotEncoder(sparse_output=False), LabelEncoder())

# Taking Fake Input data for testing our model:

In [7]:
# Example Input data for testing:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

## Encoding values of Input Data from Categorical values to Numerical values:

In [8]:
geo_data = geo_label.transform([[input_data['Geography']]])

d:\Multimedia\Programming\DeepLearning\Projects\01simpleANN\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


# Creating Data Frames:

In [9]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [10]:
geo_df = pd.DataFrame(geo_data,columns=geo_label.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
input_df['Gender'] = gender_label.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


# Concatination of One Hot Encoded Data and Input Data:

In [12]:
test_data = pd.concat([input_df.drop('Geography',axis=1),geo_df],axis=1)
test_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


# Scaling the Input Data:

In [13]:
test = scaler.transform(test_data)
test

array([[-0.53618689,  0.91174168,  0.10222793, -0.69733877, -0.25538502,
         0.80918726,  0.64750983,  0.97055459, -0.87811143,  1.00125078,
        -0.58254653, -0.57311631]])

# Prediction:

In [14]:
predict = model.predict(test) #type:ignore
predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


array([[0.08660287]], dtype=float32)

In [15]:
prediction_prob = predict[0][0]
prediction_prob

np.float32(0.08660287)

## Condition (threshold) for  Customer Leaving:

In [16]:
threshold = 0.5
if prediction_prob > 0.5:
    print("Customer is likely to Churn!")
else:
    print("Customer is not likely to Churn!")

Customer is not likely to Churn!


# Prediction Completed!